In [5]:
!pip3 install --upgrade --user --quiet google-cloud-aiplatform

In [6]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

In [1]:
PROJECT_ID = 'diesel-airfoil-423410-b3'  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

#API

In [2]:
# import module
import requests
from bs4 import BeautifulSoup
import json
# UDF for get HTML code
# from URL


def get_html(Airline_code, Flight_number, Date, Month, Year):
    def getdata(url):
        r = requests.get(url)
        return r.text

    # url
    url = "https://www.flightstats.com/v2/flight-tracker/"+Airline_code + \
        "/"+Flight_number+"?year="+Year+"&month="+Month+"&date="+Date

    # pass the url
    # into getdata function
    htmldata = getdata(url)
    soup = BeautifulSoup(htmldata, 'html.parser')
    return(soup)

# Get Flight number
# from Html code


def flight_no(soup):
    Flight_no = ""

    # Find div tag with
    # unique class name
    for i in soup.find("div", class_="ticket__FlightNumberContainer-sc-1rrbl5o-4 exbpMf"):
        Flight_no = Flight_no + (i.get_text()) + " "
    return Flight_no

# Get Airport name
# from HTML code


def airport(soup):
    Airport_name = []
    # Find div tag with
    # unique class name
    for i in soup.find_all(class_='text-helper__TextHelper-sc-8bko4a-0 cHdMkI'):
        Airport_name.append(i.get_text())
    return Airport_name

# get status
# from HTML code


def status(soup, flight_no):
  for i in soup.find_all("script"):
    if '__NEXT_DATA__' in i.get_text():
      result = i

  data = json.loads(result.string.split('{"props":')[1].split(',"page":')[0])
  status = data['initialState']['flightTracker']['flight']['status']
  departure = data['initialState']['flightTracker']['flight']['departureAirport']
  arrival = data['initialState']['flightTracker']['flight']['arrivalAirport']

  flight_details = {
    'flight': flight_no,
    'status': (status['statusDescription'], status['delay']['departure']['minutes'], status['delay']['arrival']['minutes']),
    'source': (departure['iata'], departure['name'], departure['gate'], departure['times']['scheduled']['time'], departure['times']['scheduled']['ampm'], departure['times']['estimatedActual']['time'], departure['times']['estimatedActual']['ampm']),
    'destination': (arrival['iata'], arrival['name'], arrival['gate'], arrival['times']['scheduled']['time'], arrival['times']['scheduled']['ampm'], arrival['times']['estimatedActual']['time'], arrival['times']['estimatedActual']['ampm'])
  }
  return flight_details


In [54]:
# Driver code
if __name__ == '__main__':
    # Input Data from geek
    Airline_code = 'AA'
    Flight_number = '2'
    Date = '29'
    Month = '5'
    Year = '2024'

    # Calling the get_html
    # with argument
    # function calling
    soup = get_html(Airline_code, Flight_number, Date, Month, Year)
    flight_num = flight_no(soup)
    print("Flight number : ", flight_num)
    Airport_list = airport(soup)
    result = status(soup, flight_num)
    print(result)

Flight number :  AA 2 American Airlines 
{'flight': 'AA 2 American Airlines ', 'status': ('On time', 0, 0), 'source': ('LAX', 'Los Angeles International Airport', '42A', '8:00', 'AM', '8:00', 'AM'), 'destination': ('JFK', 'John F. Kennedy International Airport', '36', '4:35', 'PM', '4:35', 'PM')}


#Model

In [4]:
import requests
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerationConfig,
    GenerativeModel,
    Part,
    Tool,
)

In [35]:
#set up function declaration
get_flight_info = FunctionDeclaration(
    name="get_flight_info",
    description="Get the flight infomation for a given airline code and flight number on a specified date",
    parameters={
        "type": "object",
        "properties": {
            "airline_code": {"type": "string", "description": "Airline Code"},
            "flight_number": {"type": "string", "description": "Flight Number"},
            "day": {"type": "string", "description": "Day"},
            "month": {"type": "string", "description": "Month"},
            "year": {"type": "string", "description": "Year"},
        },
    },
)

In [36]:
retail_tool = Tool(
    function_declarations=[
        get_flight_info
    ],
)

In [51]:
#set up agent
model = GenerativeModel(
    "gemini-1.5-pro",
    generation_config=GenerationConfig(temperature=0),
    tools=[retail_tool],
)
chat = model.start_chat()

In [52]:
#prompting
prompt = """
Do you know infomation about filght AA Airline with flight number is 2 on 27/5/2024
"""

response = chat.send_message(prompt)
response.candidates[0].content.parts[0]

function_call {
  name: "get_flight_info"
  args {
    fields {
      key: "airline_code"
      value {
        string_value: "AA"
      }
    }
    fields {
      key: "day"
      value {
        string_value: "27"
      }
    }
    fields {
      key: "flight_number"
      value {
        string_value: "2"
      }
    }
    fields {
      key: "month"
      value {
        string_value: "5"
      }
    }
    fields {
      key: "year"
      value {
        string_value: "2024"
      }
    }
  }
}

In [43]:
x = response.candidates[0].content.parts[0].function_call.args
user_prompt = {}
for key, value in zip(x.keys(), x.values()):
  print(key, value)
  user_prompt[key] = value

airline_code AA
day 27
month 5
year 2024
flight_number 2


In [45]:
soup = get_html(Airline_code = user_prompt['airline_code'], Flight_number = user_prompt['flight_number'],\
                Date = user_prompt['day'], Month = user_prompt['month'], Year = user_prompt['year'])
flight_num = flight_no(soup)
print("Flight number : ", flight_num)
Airport_list = airport(soup)
api_response = status(soup, flight_num)
print(api_response)

Flight number :  AA 2 American Airlines 
{'flight': 'AA 2 American Airlines ', 'status': ('Delayed by 34m', 0, 34), 'source': ('LAX', 'Los Angeles International Airport', '47B', '8:00', 'AM', '7:53', 'AM'), 'destination': ('JFK', 'John F. Kennedy International Airport', '43', '4:35', 'PM', '5:09', 'PM')}


In [53]:
#response
response = chat.send_message(
    Part.from_function_response(
        name="get_flight_info",
        response={
            "content": api_response,
        },
    ),
)
response.candidates[0].content.parts[0]

text: "I found a flight from LAX to JFK that matches your criteria: Flight AA 2  is delayed by 34 minutes. It departs from Los Angeles International Airport at 8:00 AM, arriving at John F. Kennedy International Airport at 5:09 PM. \n"

#Easy Function calling

In [13]:
model = GenerativeModel(
    "gemini-1.5-flash",
    generation_config=GenerationConfig(temperature=0),
    tools=[retail_tool],
)
chat = model.start_chat()

genai.GenerativeModel(
    model_name='models/gemini-1.5-flash',
    generation_config={},
    safety_settings={},
    tools=<google.generativeai.types.content_types.FunctionLibrary object at 0x7a51dc619480>,
    system_instruction=None,
)

In [9]:
chat = model.start_chat(enable_automatic_function_calling=True)

In [10]:
response = chat.send_message('I have 57 cats, each owns 44 mittens, how many mittens is that in total?')
response.text

"That's a lot of mittens! There are a total of 2508 mittens. \n"

In [11]:
for content in chat.history:
    part = content.parts[0]
    print(content.role, "->", type(part).to_dict(part))
    print('-'*80)

user -> {'text': 'I have 57 cats, each owns 44 mittens, how many mittens is that in total?'}
--------------------------------------------------------------------------------
model -> {'function_call': {'name': 'multiply', 'args': {'a': 57.0, 'b': 44.0}}}
--------------------------------------------------------------------------------
user -> {'function_response': {'name': 'multiply', 'response': {'result': 101.0}}}
--------------------------------------------------------------------------------
model -> {'text': "That's a lot of mittens! There are a total of 2508 mittens. \n"}
--------------------------------------------------------------------------------
